In [1]:
import tensorflow as tf

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
import io
import os
import re
import shutil
import string

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [11]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

84140032/84125825 [==============================] - 39s 0us/step


['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'train']

In [12]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

In [13]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [14]:
batch_size = 1024
seed = 123
train_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [15]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(5):
        print(label_batch[i].numpy(), text_batch.numpy()[i])

0 b"Oh My God! Please, for the love of all that is holy, Do Not Watch This Movie! It it 82 minutes of my life I will never get back. Sure, I could have stopped watching half way through. But I thought it might get better. It Didn't. Anyone who actually enjoyed this movie is one seriously sick and twisted individual. No wonder us Australians/New Zealanders have a terrible reputation when it comes to making movies. Everything about this movie is horrible, from the acting to the editing. I don't even normally write reviews on here, but in this case I'll make an exception. I only wish someone had of warned me before I hired this catastrophe"
1 b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams of doing to his/her school. I absolutely loved the soundtrack as well as the carefully placed cynicism. If you like monty python, You will love this film. This movie is a tad bit "grease"esk (without

In [16]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [38]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(1000, 5)

In [39]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[ 0.00961693,  0.03855425,  0.01991761, -0.04269081, -0.04625713],
       [-0.01966938,  0.0286576 ,  0.01696208, -0.0474405 ,  0.03154754],
       [ 0.00845543,  0.0269667 ,  0.0474735 , -0.02135214, -0.03707143]],
      dtype=float32)

In [40]:
result = embedding_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))
result.shape

TensorShape([2, 3, 5])

In [41]:
result.numpy()

array([[[ 0.03544592,  0.00405671,  0.01342069,  0.01754421,
         -0.00629066],
        [ 0.00961693,  0.03855425,  0.01991761, -0.04269081,
         -0.04625713],
        [-0.01966938,  0.0286576 ,  0.01696208, -0.0474405 ,
          0.03154754]],

       [[ 0.00845543,  0.0269667 ,  0.0474735 , -0.02135214,
         -0.03707143],
        [-0.02233002,  0.03010106,  0.01307045, -0.04769486,
          0.03548038],
        [ 0.00350674,  0.01174735, -0.008798  ,  0.00937635,
          0.03398216]]], dtype=float32)

In [42]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [43]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

In [44]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [45]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [46]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
20/20 [==============================] - 18s 695ms/step - loss: 0.6926 - accuracy: 0.5028 - val_loss: 0.6918 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 6s 321ms/step - loss: 0.6897 - accuracy: 0.5028 - val_loss: 0.6876 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 6s 317ms/step - loss: 0.6835 - accuracy: 0.5028 - val_loss: 0.6798 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 6s 320ms/step - loss: 0.6727 - accuracy: 0.5028 - val_loss: 0.6670 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 6s 307ms/step - loss: 0.6556 - accuracy: 0.5028 - val_loss: 0.6474 - val_accuracy: 0.4886
Epoch 6/15
20/20 [==============================] - 6s 315ms/step - loss: 0.6308 - accuracy: 0.5028 - val_loss: 0.6220 - val_accuracy: 0.4886
Epoch 7/15
20/20 [==============================] - 6s 322ms/step - loss: 0.6006 - accuracy: 0.5203 - val_loss: 0.5929 - val_accuracy: 0.5584
Epoch

In [47]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,289
Trainable params: 160,289
Non-trai

In [48]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir logs


In [49]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [50]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index == 0:
        continue  # skip 0, it's padding.
    vec = weights[index]
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")
    
out_v.close()
out_m.close()